# 00-2 Preprocessing for Monthly Transactions

In this notebook, we split the transaction into disjoint monthly intervals from August 2019 to September 2020.

In [1]:
import pandas as pd
import numpy as np
from datetime import date

- We start by reading in the transaction log from a csv file, setting ```t_dat``` to a datetime column.

In [3]:
transactions = pd.read_csv('../data/transactions_cleaned.csv', date_parser='t_dat')

In [4]:
articles = pd.read_csv('../data/articles.csv')

In [5]:
transactions.head()

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [6]:
# clean up the dataframe a bit
transactions.drop(columns='Unnamed: 0', inplace=True)

In [7]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [8]:
transactions = transactions[['t_dat', 'customer_id', 'article_id']]

In [9]:
transactions.head()

,t_dat,customer_id,article_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004


- Next we'll merge the dataframes so that we have transactions by datetime and product code.

In [10]:
transactions = transactions.merge(right=articles[['article_id', 'product_code']], how='left', on='article_id')

In [11]:
transactions = transactions[['t_dat', 'product_code']]

In [12]:
transactions.tail()

,t_dat,product_code
31651673,2020-09-22,929511
31651674,2020-09-22,891322
31651675,2020-09-22,918325
31651676,2020-09-22,833459
31651677,2020-09-22,898573


- Now we can filter transactions by date, like so:

In [13]:
transactions = transactions[ transactions['t_dat'] > '2019-08']

In [14]:
transactions

,t_dat,product_code
14782429,2019-08-01,715624
14782430,2019-08-01,803250
14782431,2019-08-01,787883
14782432,2019-08-01,787883
14782433,2019-08-01,733097
...,...,...
31651673,2020-09-22,929511
31651674,2020-09-22,891322
31651675,2020-09-22,918325
31651676,2020-09-22,833459


In [15]:
transactions[ (transactions['t_dat'] > '2019-09') & (transactions['t_dat'] < '2019-10')]

,t_dat,product_code
16031743,2019-09-01,727880
16031744,2019-09-01,767869
16031745,2019-09-01,717490
16031746,2019-09-01,547780
16031747,2019-09-01,803969
...,...,...
17256883,2019-09-30,764998
17256884,2019-09-30,683662
17256885,2019-09-30,595697
17256886,2019-09-30,160442


- We now loop through each month, slicing the dataframe by month and saving each slice to a csv.

In [16]:
def get_transactions(df, datetime_col, start, end, filepath='temp.csv'):
    df[ (df[datetime_col] > start) & (df[datetime_col] < end) ].to_csv(filepath, index=False)
    

In [17]:
get_transactions(transactions, datetime_col = 't_dat', start= '2019-08', end='2019-09',filepath='../data/transactions_2019-08.csv') 

In [18]:
months = ['2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02',
          '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09']

In [19]:
for i in range(len(months)-1):
    get_transactions(transactions, datetime_col = 't_dat', start=months[i], end=months[i+1], filepath = '../data/transactions_'+months[i]+'.csv')

In [20]:
type(current_month)

str

In [21]:
transactions.value_counts(subset='product_code')[0:10]

product_code
706016    125284
562245     80168
599580     78703
751471     61971
717490     53158
783346     51563
720125     49894
759871     48815
695632     45024
778064     44376
dtype: int64

In [22]:
list(transactions.value_counts(subset='product_code').index)[0]

706016

- Now we can filter the data set by time intervals to get a "Top Sellers" list by Month, Week, and even Day.

In [23]:
transactions[ (transactions['t_dat'] > '2020-05') & (transactions['t_dat'] < '2020-06') ].value_counts(subset='product_code')[0:10]

product_code
599580    13533
684209     9290
688537     8019
854683     6825
706016     6592
554598     6358
854677     6168
776237     6081
685816     6063
759871     6042
dtype: int64